In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
from sklearn.pipeline import make_pipeline

In [3]:
import gensim

считываем вопросы mail.ru из кулинарных категорий

In [4]:
question = pd.read_csv('merged.csv', usecols=['question', 'category_urlname']).dropna()

In [5]:
food_categories = ['sweets_baking', 'drinks', 'food_devices', 'main_courses',
       'food_other', 'celebration', 'food_howto', 'food_buy', 'salads',
       'food_preserving', 'soups', 'food_kid']
questions = question.question
food_questions = question[np.in1d(question.category_urlname, food_categories)].question

используем лучший классификатор для выделение кулинарии из лога с самыми частотными запросами

In [6]:
log = list(map(str.strip, open('./data/uniq_queries_100', 'r').readlines()))


In [7]:
from sklearn.pipeline import make_pipeline

In [8]:
vectorizer = pickle.load(open('vectorizers/unigram.pkl', 'rb'))
log_ohe = vectorizer.transform(log)

In [9]:
model = pickle.load(open('models/unigram_lr.pkl', 'rb'))

In [10]:
pipeline = make_pipeline(vectorizer, model)


In [11]:
food_log = np.array(log)[np.nonzero(model.predict(log_ohe))[0]]

In [12]:
food_queries = []

In [21]:
%%time 
big_log = pd.read_csv('data/kak-frequencies-sorted-kak.tsv',sep='\t', chunksize=1000000, header=None)
j = 0
for i in big_log:
    j += 1
    print(j)    

    predicts = pipeline.predict(i[0])
    food_queries.extend(list(i[0].values[predicts == 1]))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
CPU times: user 49min 55s, sys: 27.3 s, total: 50min 22s
Wall time: 51min 20s


In [22]:
F = open('data/food_queries.txt', 'w')
for sent in food_queries:
    F.write(sent+'\n')
F.close()

In [16]:
food_queries[1]

'как варить рис'

In [39]:
len(food_queries)

16353643

In [49]:
from nltk.tokenize import WordPunctTokenizer
from gensim.models import Phrases
Phrases.

In [13]:
class Generator(object):
    def __init__(self, data):
        self.data = data
        self.tokenizer = WordPunctTokenizer()

    def __iter__(self):
        for row in self.data.values:
           yield self.tokenizer.tokenize(row.lower())

In [14]:
corpus = Generator(food_questions)

In [54]:
model_w2v = gensim.models.Word2Vec(min_count=1)
%time model_w2v.build_vocab(corpus)
%time model_w2v.train(corpus, total_examples=model_w2v.corpus_count, epochs=10)

CPU times: user 9.39 s, sys: 4.06 ms, total: 9.4 s
Wall time: 9.4 s
CPU times: user 2min 53s, sys: 1.6 s, total: 2min 55s
Wall time: 1min 36s


(29596954, 39580870)

In [56]:
model_w2v.save(open('wv2model_mailru.vw', 'wb'))

In [59]:
model_w2v = gensim.models.Word2Vec.load('wv2model_mailru.vw')

In [65]:
corpus_log = Generator(pd.Series(food_queries))

In [66]:
model_w2v_big = gensim.models.Word2Vec(min_count=1)
%time model_w2v_big.build_vocab(corpus_log)
%time model_w2v_big.train(corpus_log, total_examples=model_w2v_big.corpus_count, epochs=10)

CPU times: user 2min 52s, sys: 1.07 s, total: 2min 53s
Wall time: 2min 53s
CPU times: user 1h 15min 21s, sys: 41.1 s, total: 1h 16min 2s
Wall time: 54min 45s


(672249086, 945891650)

In [77]:
model_w2v_big.most_similar('бензин')

/home/mikhail/.conda/envs/work/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('kddfylljdjt', 0.7110490798950195),
 ('бензина', 0.708771288394928),
 ('двигателеsubaru', 0.7043659090995789),
 ('бинзин', 0.7010259628295898),
 ('аплепиховые', 0.6875290274620056),
 ('присадку', 0.6750895977020264),
 ('заборник', 0.673613429069519),
 ('atf', 0.67183518409729),
 ('промывочное', 0.6628495454788208),
 ('сивушное', 0.6616705656051636)]

In [78]:
model_w2v_big.save('wv2model_alllog.vw')

In [4]:
model_2 = gensim.models.Word2Vec.load('wv2model_mailru.vw')

In [5]:
model_2.wv.save_word2vec_format('vectors.txt')

In [4]:
f = open('./vectors.txt', 'r')
f.readline()
vectors = list(map(str.strip,f.readlines()))
meta = []
wvs = []
for v in vectors:
    word = v.split()[0]
    vector = ' '.join(v.split()[1:])
    meta.append(word)
    wvs.append(vector)

In [2]:
wvs2 = []
for vector in wvs:
    wvs2.append(list(map(int, vector.split())))
wvs2

NameError: name 'wvs' is not defined

In [22]:
f = open('../../Загрузки/puzer.github.io-master/projector/oss_data/vectors.txt', 'w')
for vector in wvs:
    f.write(vector.replace(' ', '\t') + "\n")
f.close()

In [13]:
f = open('meta.txt', 'w')
for vector in meta:
    f.write(vector + "\n")
f.close()

In [18]:
a = map(str.split, wvs)
b = []
for v in a:
    v = list(map(float, v))
    b.append(v)

(132524, 100)